# Descriptors

Descriptors are a specific Python feature that power a lot of the magic hidden under the language’s hood. If you’ve ever thought that Python descriptors are an advanced topic with few practical applications, then this tutorial is the perfect tool to help you understand this powerful feature. You’ll come to understand why Python descriptors are such an interesting topic, and what kind of use cases you can apply them to.

## What Are Python Descriptors?

Descriptors are Python objects that implement a method of the descriptor protocol, which gives you the ability to create objects that have special behavior when they’re accessed as attributes of other objects. They are a general-purpose way of intercepting attribute access. Descriptors are the mechanism behind properties' static methods, class methods, super methods, etc.

Here you can see the correct definition of the descriptor protocol:


```python
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None
__delete__(self, obj) -> None
__set_name__(self, owner, name)
```

If your descriptor implements just `.__get__()`, then it’s said to be a **non-data descriptor**. If it implements `.__set__()` or `.__delete__()`, then it’s said to be a **data descriptor**. Note that this difference is not just about the name, but it’s also a difference in behavior. The non-data descriptors are only readable while the data descriptors are both readable and writable. That’s because data descriptors have precedence during the lookup process, as you’ll see later on.

It is important to note that descriptors are assigned to a class, not to the instance of a class. Modifying the class overwrites or deletes the descriptor itself, rather than triggering its code.

Finally, the descriptor class is not only confined to have these three methods, which means it can also contain any other attribute apart from the `get`, `set`, and `delete` method.

Let's understand the get, set, and delete methods in more detail inspired by this IBM Developer page:

- `self` is the instance of the descriptor you create (Real Python).
- `object` is the instance of the object your descriptor is attached (Real Python).
- `type` is the type of the object the descriptor is attached to (Real Python).
- `value` is the value that is assigned to the attribute of the descriptor. get(self, object, type) set(self, object, value) delete(self, object)
- `__get__()` accesses the attribute or when you want to extract some information. It returns the value of the attribute or raises the AttributeError exception if a requested attribute is not present.

- `__set__()` is called in an attribute assignment operation that sets the value of an attribute. Returns nothing. But can raise the AttributeError exception.

- `__delete__()` controls a delete operation, i.e., when you would want to delete the attribute from an object. Returns nothing.

## Purpose of Descriptors

Let's define a class car that has three attributes, namely `brand` and `fuel_cap`. You will use the `__init__()` method to initialize the attributes of the class. Then, you will use the magic function `__str__()`, which will simply return the output of the three attributes that you will pass to the class while creating the object.

In [1]:
class Car:
    def __init__(self, brand, fuel_cap):
        self.brand = brand
        self.fuel_cap = fuel_cap

    def __str__(self):
        return f"{self.brand} model with a fuel capacity of {self.fuel_cap} ltr."

In [3]:
car_2 = Car("BMW", 40)
print(car_2)

BMW model with a fuel capacity of 40 ltr.


So as you can see from the above output, everything looks great!

Now let's change the fuel capacity of the car to negative 40.

In [7]:
car_2 = Car("BMW", -40)
print(car_2)

BMW model with a fuel capacity of -40 ltr.


Hold on, there is something wrong, isn't it? The fuel capacity of the car can never be negative. However, Python accepts it as an input without any error. That's because Python is a dynamic programming language that does not support type-checking explicitly.

To avoid this issue, let's add an `if` condition in the `__init__()` method and check whether the inputted fuel capacity is valid or invalid. If the fuel capacity entered is invalid, then raise a ValueError exception.

In [10]:
class Car:
    def __init__(self, brand, fuel_cap):
        self.brand = brand
        self.fuel_cap = fuel_cap
        
        if self.fuel_cap <= 0:
            raise ValueError("Fuel capacity can never be less than zero!")

    def __str__(self):
        return f"{self.brand} model with a fuel capacity of {self.fuel_cap} ltr."

In [11]:
car_2 = Car("BMW", -40)
print(car_2)

ValueError: Fuel capacity can never be less than zero!

From the above output, you can observe that everything works fine for now since the program raises a `ValueError` if the fuel capacity is below zero.

However, what if you would like to change the fuel capacity attribute to negative 40 explicitly later on. In this case, it will not work, since the type-checking will be done only in the `__init__()` method once. As you would know, the `__init__()` method is a constructor, and it is called only once when you create an object of the class. Hence, the custom type-checking will fail later on.

Let's understand it with an example.

In [13]:
class Car:
    def __init__(self, brand, fuel_cap):
        self.brand = brand
        self.fuel_cap = fuel_cap
        
        if self.fuel_cap <= 0:
            raise ValueError("Fuel capacity can never be less than zero!")

    def __str__(self):
        return f"{self.brand} model with a fuel capacity of {self.fuel_cap} ltr."

In [14]:
car_2 = Car("BMW", 40)
print(car_2)

BMW model with a fuel capacity of 40 ltr.


In [15]:
car_2.fuel_cap = -40

And there you go! You were able to break out of type-checking.

Now think it this way, what if you have many other attributes of the car like `mileage`, `price`, `accessories`, etc. which requires type-checking as well and you also would like functionality in which some of these attributes have only `read` access. Wouldn't that be so annoying?

Well, to address all of the above problems, Python has Descriptors that come to the rescue!

As you learned above that any class that implements `__get__()`, `__set()__`, or `__delete()__` magic methods for an object of descriptor protocol are called Descriptors. They also give you additional control over how an attribute should work like whether it should have a read or write access.

Now let's extend the above example by adding the Python Descriptor methods.

In [7]:
class Descriptor:
    def __init__(self):
        self.__fuel_cap = 0

    def __get__(self, instance, owner):
        print("Calling get...")
        return self.__fuel_cap

    def __set__(self, instance, value):
        print("Calling set...")
        if not isinstance(value, int):
            raise TypeError("Fuel Capacity can only be an integer")

        if value < 0:
            raise ValueError("Fuel Capacity can never be less than zero")

        self.__fuel_cap = value

    def __delete__(self, instance):
        print("Calling delete...")
        del self.__fuel_cap

In [8]:
class Car:
    fuel_cap = Descriptor()

    def __init__(self, brand, fuel_cap):
        self.brand = brand
        self.fuel_cap = fuel_cap

    def __str__(self):
        return f"{self.brand} model with a fuel capacity of {self.fuel_cap} ltr."

In [9]:
car_2 = Car("BMW", 40)

Calling set...


In [95]:
print(car_2)

---> <class 'type'>
Calling get...
BMW model with a fuel capacity of 40 ltr.


Perfect! So as you can see, it works when you update the fuel capacity attribute later on.

Well, there is a slight problem in Descriptors and which is that when you create a new instance or a second instance of the class, the previous instance value gets overridden. The reason is that Descriptors are linked to class and not the instance.

In [76]:
car_2.fuel_cap

Calling get...


40

In [77]:
car_3 = Car("BMW", 80)

Calling set...


In [78]:
car_2.fuel_cap

Calling get...


80

### Dynamic Lookup

Interesting descriptors typically run computations instead of returning constants:

In [10]:
from pathlib import Path

class DirectorySize:

    def __get__(self, instance, owner):
        return len(list(Path(instance.dirname).iterdir()))

class Directory:

    size = DirectorySize()              # Descriptor instance

    def __init__(self, dirname):
        self.dirname = dirname          # Regular instance attribute

In [11]:
s = Directory('.')

In [12]:
s.size

12

In [13]:
s.dirname = '..'

In [14]:
s.size

11

Besides showing how descriptors can run computations, this example also reveals the purpose of the parameters to `__get__()`. The self parameter is size, an instance of DirectorySize. The obj parameter is either g or s, an instance of Directory. It is the obj parameter that lets the `__get__()` method learn the target directory. The objtype parameter is the class Directory.

### Managed attributes

A popular use for descriptors is managing access to instance data. The descriptor is assigned to a public attribute in the class dictionary while the actual data is stored as a private attribute in the instance dictionary. The descriptor’s `__get__()` and `__set__()` methods are triggered when the public attribute is accessed.

In the following example, `age` is the public attribute and `_age` is the private attribute. When the public attribute is accessed, the descriptor logs the lookup or update:

In [141]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info(f"Accessing 'age' giving {value}")
        return value

    def __set__(self, obj, value):
        logging.info(f"Updating 'age' to {value}")
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

An interactive session shows that all access to the managed attribute age is logged, but that the regular attribute name is not logged:m

In [142]:
ali = Person("Ali", 28)

INFO:root:Updating 'age' to 28


In [143]:
vars(ali)

{'name': 'Ali', '_age': 28}

In [144]:
ali.age = 29

INFO:root:Updating 'age' to 29


In [145]:
ali.name

'Ali'

In [146]:
ali.age

INFO:root:Accessing 'age' giving 29


29

One major issue with this example is that the private name `_age` is hardwired in the `LoggedAgeAccess` class. That means that each instance can only have one logged attribute and that its name is unchangeable. In the next example, we’ll fix that problem.

### Customized names

When a class uses descriptors, it can inform each descriptor about which variable name was used.

In this example, the `Person` class has two descriptor instances, name and age. When the `Person` class is defined, it makes a callback to `__set_name__()` in `LoggedAccess` so that the field names can be recorded, giving each descriptor its own public name and private name:

In [166]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info(f"Accessing '{self.public_name}' giving {value}")
        return value

    def __set__(self, obj, value):
        logging.info(f"Updating '{self.public_name}' to {value}")
        setattr(obj, self.private_name, value)

class Person:

    name = LoggedAccess()                # First descriptor instance
    age = LoggedAccess()                 # Second descriptor instance

    def __init__(self, name, age):
        self.name = name                 # Calls the first descriptor
        self.age = age                   # Calls the second descriptor

    def birthday(self):
        self.age += 1

<class '__main__.Person'> name
<class '__main__.Person'> age


In [163]:
ali = Person("Ali", 28)

INFO:root:Updating 'name' to Ali
INFO:root:Updating 'age' to 28


In [164]:
ali.name

INFO:root:Accessing 'name' giving Ali


'Ali'

In [167]:
vars(ali)

{'_name': 'Ali', '_age': 28}

### Closing thoughts
A descriptor is what we call any object that defines `__get__()`, `__set__()`, or `__delete__()`.

- Optionally, descriptors can have a `__set_name__()` method. This is only used in cases where a descriptor needs to know either the class where it was created or the name of class variable it was assigned to. (This method, if present, is called even if the class is not a descriptor.)

- Descriptors get invoked by the dot “operator” during attribute lookup. If a descriptor is accessed indirectly with `vars(some_class)[descriptor_name]`, the descriptor instance is returned without invoking it.

- Descriptors only work when used as class variables. When put in instances, they have no effect.

- The main motivation for descriptors is to provide a hook allowing objects stored in class variables to control what happens during attribute lookup.

- Traditionally, the calling class controls what happens during lookup. Descriptors invert that relationship and allow the data being looked-up to have a say in the matter.

- Descriptors are used throughout the language. It is how functions turn into bound methods. Common tools like `classmethod()`, `staticmethod()`, `property()`, and `functools.cached_property()` are all implemented as descriptors.

## Complete Practical Example

In this example, we create a practical and powerful tool for locating notoriously hard to find data corruption bugs.

### Validator class

A validator is a descriptor for managed attribute access. Prior to storing any data, it verifies that the new value meets various type and range restrictions. If those restrictions aren’t met, it raises an exception to prevent data corruption at its source.

This `Validator` class is both an abstract base class and a managed attribute descriptor:

In [19]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

Custom validators need to inherit from `Validator` and must supply a `validate()` method to test various restrictions as needed.

### Custom validators
Here are three practical data validation utilities:

- `OneOf` verifies that a value is one of a restricted set of options.

- `Number` verifies that a value is either an `int` or `float`. Optionally, it verifies that a value is between a given minimum or maximum.

- `String` verifies that a value is a `str`. Optionally, it validates a given minimum or maximum length. It can validate a user-defined predicate as well.

In [20]:
class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

In [21]:
class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )

In [22]:
class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

### Practical application

Here’s how the data validators can be used in a real class:

In [38]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

The descriptors prevent invalid instances from being created:

In [39]:
# this should raise `ValueError`
Component(name='Widget', kind='metal', quantity=5)

ValueError: Expected <method 'isupper' of 'str' objects> to be true for 'Widget'

In [40]:
# this should raise `ValueError`
Component(name='WIDGET', kind='metle', quantity=5)

ValueError: Expected 'metle' to be one of {'plastic', 'wood', 'metal'}

In [41]:
# this should raise `ValueError`
Component(name='WIDGET', kind='metal', quantity=-5)

ValueError: Expected -5 to be at least 0

In [42]:
# this should run successfully with no error
c = Component('WIDGET', 'metal', 5)

### Pure Python Equivalents

The descriptor protocol is simple and offers exciting possibilities. Several use cases are so common that they have been prepackaged into built-in tools. Properties, bound methods, static methods, class methods, and __slots__ are all based on the descriptor protocol.

Calling `property()` is a succinct way of building a data descriptor that triggers a function call upon access to an attribute. Its signature is:

```python
property(fget=None, fset=None, fdel=None, doc=None) -> property
```

The documentation shows a typical use to define a managed attribute `x`:

In [182]:
class C:
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = property(getx, setx, delx, "I'm the 'x' property.")

To see how ‍`property()` is implemented in terms of the descriptor protocol, here is a pure Python equivalent:

In [70]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

The following example uses a property that logs a message to the console when it’s accessed:

In [101]:
# property_decorator.py
class Foo():
    def __init__(self, value: int):
        self._value = value
    
    @property
    def value(self):
        print("accessing the attribute to get the value")
        return self._value

    @value.setter
    def value(self, value):
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")
        
    @value.deleter
    def value(self):
        del self._value

In [203]:
obj = Foo(20)
obj.value

accessing the attribute to get the value


20

The example above makes use of decorators to define a property, but as you may know, decorators are just syntactic sugar. The example before, in fact, can be written as follows:

In [100]:
# property_decorator.py
class Foo():
    def __init__(self, value):
        self._value = value
    
    def getter(self):
        print("accessing the attribute to get the value")
        return self._value

    def setter(self, value):
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

    def deleter(self):
        print("deleting the attribute")
        del self._value

    value = property(getter, setter, deleter)

In [211]:
obj = Foo(20)
obj.value

accessing the attribute to get the value


20

In [212]:
del obj.value

deleting the attribute


## Python Descriptors in Methods and Functions
If you’ve ever written an object-oriented program in Python, then you’ve certainly used methods. These are regular functions that have the first argument reserved for the object instance. When you access a method using dot notation, you’re calling the corresponding function and passing the object instance as the first parameter.

The magic that transforms your `obj.method(*args)` call into `method(obj, *args)` is inside a `.__get__()` implementation of the `function` object that is, in fact, a **non-data descriptor**. In particular, the `function` object implements `.__get__()` so that it returns a bound method when you access it with dot notation. The (`*args`) that follow invoke the functions by passing all the extra arguments needed.

To get an idea for how it works, take a look at this pure Python example from the official docs:

In [213]:
import types

class Function(object):
    ...
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return types.MethodType(self, obj)

In [102]:
def add(a, b):
    return a + b

In the example above, when the function is accessed with dot notation, `.__get__()` is called and a bound method is returned.

This works for regular instance methods just like it does for class methods or static methods. So, if you call a static method with `obj.method(*args)`, then it’s automatically transformed into `method(*args)`. Similarly, if you call a class method with `obj.method(type(obj), *args)`, then it’s automatically transformed into `method(type(obj), *args)`.

In the official docs, you can find some examples of how static methods and class methods would be implemented if they were written in pure Python instead of the actual C implementation. For instance, a possible static method implementation could be this:

In [214]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

Likewise, this could be a possible class method implementation:

In [ ]:
class Student:
    
    @classmethod
    def classmethod_(cls, ):
        pass

In [216]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc

Note that, in Python, a class method is just a static method that takes the class reference as the first argument of the argument list.

## How Attributes Are Accessed With the Lookup Chain

To understand a little more about Python descriptors and Python internals, you need to understand what happens in Python when an attribute is accessed. In Python, every object has a built-in `__dict__` attribute. This is a dictionary that contains all the attributes defined in the object itself. To see this in action, consider the following example:

In [218]:
class Vehicle():
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

In [219]:
my_car = Car("red")
print(my_car.__dict__)
print(type(my_car).__dict__)

{'color': 'red'}
{'__module__': '__main__', 'number_of_weels': 4, '__init__': <function Car.__init__ at 0x7fb65beb5b90>, '__doc__': None}


This code creates a new object and prints the contents of the `__dict__` attribute for both the object and the class. Now, run the script and analyze the output to see the `__dict__` attributes set:

```bash
{'color': 'red'}
{'__module__': '__main__', 'number_of_weels': 4, '__init__': <function Car.__init__ at 0x10fdeaea0>, '__doc__': None}
```

The `__dict__` attributes are set as expected. Note that, in Python, **everything is an object**. A class is actually an object as well, so it will also have a `__dict__` attribute that contains all the attributes and methods of the class.

In [221]:
# lookup.py
class Vehicle(object):
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

my_car = Car("red")

In this example, you create an instance of the `Car` class that inherits from the `Vehicle` class. Then, you access some attributes. If you run this example, then you can see that you get all the values you expect.

In [223]:
print(my_car.color)
print(my_car.number_of_weels)
print(my_car.can_fly)

red
4
False


Here, when you access the attribute `color` of the instance `my_car`, you’re actually accessing a single value of the `__dict__` attribute of the object `my_car`. When you access the attribute `number_of_wheels` of the object `my_car`, you’re really accessing a single value of the `__dict__` attribute of the class `Car`. Finally, when you access the `can_fly` attribute, you’re actually accessing it by using the `__dict__` attribute of the `Vehicle` class.

This means that it’s possible to rewrite the above example like this:

In [128]:
# lookup2.py
class Vehicle():
    can_fly = False
    number_of_wheels = 0

class Car(Vehicle):
    number_of_wheels = 4

    def __init__(self, color):
        self.color = color

In [140]:
my_car = Car("red")

When you test this new example, you should get the same result:

In [227]:
print(my_car.__dict__['color'])
print(type(my_car).__dict__['number_of_weels'])
print(type(my_car).__base__.__dict__['can_fly'])

red
4
False


So, what happens when you access the attribute of an object with dot notation? How does the interpreter know what you really need? Well, here’s where a concept called the **lookup chain** comes in:

1. First, you’ll get the result returned from the `__get__` method of the **data descriptor** named after the attribute you’re looking for.

2. If that fails, then you’ll get the value of your object’s `__dict__` for the key named after the attribute you’re looking for.

3. If that fails, then you’ll get the result returned from the `__get__` method of the **non-data descriptor** named after the attribute you’re looking for.

4. If that fails, then you’ll get the value of your object type’s `__dict__` for the key named after the attribute you’re looking for.

5. If that fails, then you’ll get the value of your object parent type’s `__dict__` for the key named after the attribute you’re looking for.

6. If that fails, then the previous step is repeated for all the parent’s types in the method resolution order of your object.

7. If everything else has failed, then you’ll get an `AttributeError` exception.

Now you can see why it’s important to know if a descriptor is a data descriptor or a non-data descriptor? They’re on different levels of the lookup chain, and you’ll see later on that this difference in behavior can be very convenient.

## Why Use Python Descriptors?

Now you know what Python descriptors are and how Python itself uses them to power some of its features, like methods and properties. You’ve also seen how to create a Python descriptor while avoiding some common pitfalls. Everything should be clear now, but you may still wonder why you should use them.

In my experience, I’ve known a lot of advanced Python developers that have never used this feature before and that have no need for it. That’s quite normal because there are not many use cases where Python descriptors are necessary. However, that doesn’t mean that Python descriptors are just an academic topic for advanced users. There are still some good use cases that can justify the price of learning how to use them.

### Lazy Properties

The first and most straightforward example is **lazy properties**. These are properties whose initial values are not loaded until they’re accessed for the first time. Then, they load their initial value and keep that value cached for later reuse.

Consider the following example. You have a class `DeepThought` that contains a method `meaning_of_life()` that returns a value after a lot of time spent in heavy concentration:

In [142]:
# slow_properties.py
import time

class DeepThought:
    def meaning_of_life(self):
        time.sleep(3)
        return 42

In [143]:
my_deep_thought_instance = DeepThought()

In [144]:
print(my_deep_thought_instance.meaning_of_life())

42


In [145]:
print(my_deep_thought_instance.meaning_of_life())

42


In [146]:
print(my_deep_thought_instance.meaning_of_life())

42


If you run this code and try to access the method three times, then you get an answer every three seconds, which is the length of the sleep time inside the method.

Now, a lazy property can instead evaluate this method just once when it’s first executed. Then, it will cache the resulting value so that, if you need it again, you can get it in no time. You can achieve this with the use of Python descriptors:

In [159]:
# lazy_properties.py
import time

class LazyProperty:
    def __init__(self, function):
        self.function = function
        self.name = function.__name__

    def __get__(self, obj, type=None) -> object:
        obj.__dict__[self.name] = self.function(obj)
        return obj.__dict__[self.name]

class DeepThought:
    @LazyProperty
    def meaning_of_life(self):
        time.sleep(3)
        return 42

In [160]:
my_deep_thought_instance = DeepThought()

In [161]:
vars(my_deep_thought_instance)

{}

In [162]:
print(my_deep_thought_instance.meaning_of_life)

42


In [163]:
vars(my_deep_thought_instance)

{'meaning_of_life': 42}

In [164]:
print(my_deep_thought_instance.meaning_of_life)

42


In [165]:
print(my_deep_thought_instance.meaning_of_life)

42


Take your time to study this code and understand how it works. Can you see the power of Python descriptors here? In this example, when you use the `@LazyProperty` descriptor, you’re instantiating a descriptor and passing to it `.meaning_of_life()`. This descriptor stores both the method and its name as instance variables.

Since it is a non-data descriptor, when you first access the value of the `meaning_of_life` attribute, `.__get__()` is automatically called and executes `.meaning_of_life()` on the `my_deep_thought_instance` object. The resulting value is stored in the `__dict__` attribute of the object itself. When you access the `meaning_of_life` attribute again, Python will use the **lookup chain** to find a value for that attribute inside the `__dict__` attribute, and that value will be returned immediately.

Note that this works because, in this example, you’ve only used one method `.__get__()` of the descriptor protocol. You’ve also implemented a non-data descriptor. If you had implemented a data descriptor, then the trick would not have worked. Following the lookup chain, it would have had precedence over the value stored in `__dict__.` To test this out, run the following code:

In [238]:
# wrong_lazy_properties.py
import time

class LazyProperty:
    def __init__(self, function):
        self.function = function
        self.name = function.__name__

    def __get__(self, obj, type=None) -> object:
        obj.__dict__[self.name] = self.function(obj)
        return obj.__dict__[self.name]

    def __set__(self, obj, value):
        pass

class DeepThought:
    @LazyProperty
    def meaning_of_life(self):
        time.sleep(3)
        return 42

In [239]:
my_deep_thought_instance = DeepThought()

In [240]:
print(my_deep_thought_instance.meaning_of_life)

42


In [241]:
print(my_deep_thought_instance.meaning_of_life)

42


In [242]:
print(my_deep_thought_instance.meaning_of_life)

42


In this example, you can see that just implementing `.__set__()`, even if it doesn’t do anything at all, creates a data descriptor. Now, the trick of the lazy property stops working.

### D.R.Y. Code

Another typical use case for descriptors is to write reusable code and make your code [D.R.Y.](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) Python descriptors give developers a great tool to write reusable code that can be shared among different properties or even different classes.

Consider an example where you have five different properties with the same behavior. Each property can be set to a specific value only if it’s an even number. Otherwise, it’s value is set to 0:

In [243]:
# properties.py
class Values:
    def __init__(self):
        self._value1 = 0
        self._value2 = 0
        self._value3 = 0
        self._value4 = 0
        self._value5 = 0

    @property
    def value1(self):
        return self._value1

    @value1.setter
    def value1(self, value):
        self._value1 = value if value % 2 == 0 else 0

    @property
    def value2(self):
        return self._value2

    @value2.setter
    def value2(self, value):
        self._value2 = value if value % 2 == 0 else 0

    @property
    def value3(self):
        return self._value3

    @value3.setter
    def value3(self, value):
        self._value3 = value if value % 2 == 0 else 0

    @property
    def value4(self):
        return self._value4

    @value4.setter
    def value4(self, value):
        self._value4 = value if value % 2 == 0 else 0

    @property
    def value5(self):
        return self._value5

    @value5.setter
    def value5(self, value):
        self._value5 = value if value % 2 == 0 else 0

In [245]:
my_values = Values()
my_values.value1 = 1
my_values.value2 = 4

In [246]:
print(my_values.value1)
print(my_values.value2)

0
4


As you can see, you have a lot of duplicated code here. It’s possible to use Python descriptors to share behavior among all the properties. You can create an EvenNumber descriptor and use it for all the properties like this:

In [247]:
# properties2.py
class EvenNumber:
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = (value if value % 2 == 0 else 0)

class Values:
    value1 = EvenNumber()
    value2 = EvenNumber()
    value3 = EvenNumber()
    value4 = EvenNumber()
    value5 = EvenNumber()

In [248]:
my_values = Values()
my_values.value1 = 1
my_values.value2 = 4
print(my_values.value1)
print(my_values.value2)

0
4


This code looks a lot better now! The duplicates are gone and the logic is now implemented in a single place so that if you need to change it, you can do so easily.

## Conclusion
Now that you know how Python uses descriptors to power some of its great features, you’ll be a more conscious developer who understands why some Python features have been implemented the way they are.

You’ve learned:

- What Python descriptors are and when to use them
- Where descriptors are used in Python’s internals
- How to implement your own descriptors